In [ ]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [ ]:
training_data=datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)
test_data=datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)


100%|██████████| 26.4M/26.4M [00:02<00:00, 9.82MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 169kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.82MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 12.6MB/s]


In [ ]:
batch_size=64
train_dataloader=DataLoader(training_data, batch_size=batch_size)
test_dataloader=DataLoader(test_data, batch_size=batch_size)

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
class NeuralNetworks (nn.Module):
    def __init__(self):
        super(NeuralNetworks, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack= nn. Sequential(
            nn. Linear(28*28, 512),
            nn.ReLU(),
            nn. Linear(512, 512),
            nn.ReLU(),
            nn. Linear(512, 10)
        )
    def forward(self,x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

In [ ]:
model= NeuralNetworks().to(device)
print(model)

NeuralNetworks(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn=nn.CrossEntropyLoss()
optimizer= torch.optim.SGD(model.parameters(),lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size=len(dataloader.dataset)
  model.train()
  for batch, (X,Y) in enumerate(dataloader):
    X,Y=X.to(device),Y.to(device)

    pred=model(X)
    loss=loss_fn(pred,Y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current= loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
  size=len(dataloader.dataset)
  num_batches=len(dataloader)
  model.eval()
  test_loss, correct=0,0
  with torch.no_grad():
    for X,y in dataloader:
      X,y=X.to(device),y.to(device)
      pred=model(X)
      test_loss += loss_fn(pred,y).item()
      correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs =20
for t in range(epochs):
  print(f"Epoch{t+1}\n---------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch1
---------------------------
loss: 2.300239 [    0/60000]
loss: 2.298120 [ 6400/60000]
loss: 2.276182 [12800/60000]
loss: 2.268623 [19200/60000]
loss: 2.260895 [25600/60000]
loss: 2.224757 [32000/60000]
loss: 2.238086 [38400/60000]
loss: 2.200205 [44800/60000]
loss: 2.193989 [51200/60000]
loss: 2.169950 [57600/60000]
Test Error: 
 Accuracy: 42.1%, Avg loss: 2.159010 

Epoch2
---------------------------
loss: 2.168904 [    0/60000]
loss: 2.165533 [ 6400/60000]
loss: 2.105788 [12800/60000]
loss: 2.117523 [19200/60000]
loss: 2.076804 [25600/60000]
loss: 2.015951 [32000/60000]
loss: 2.048612 [38400/60000]
loss: 1.970818 [44800/60000]
loss: 1.970886 [51200/60000]
loss: 1.901646 [57600/60000]
Test Error: 
 Accuracy: 58.2%, Avg loss: 1.895533 

Epoch3
---------------------------
loss: 1.928545 [    0/60000]
loss: 1.905715 [ 6400/60000]
loss: 1.783320 [12800/60000]
loss: 1.818593 [19200/60000]
loss: 1.722906 [25600/60000]
loss: 1.670426 [32000/60000]
loss: 1.698788 [38400/60000]
loss: 1.

In [ ]:
torch.save(model.state_dict(),"model.pth")
print("Model Saved!")

Model Saved!
